In [ ]:
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
import pandas as pd
import numpy as np
from sklearn import datasets, model_selection, metrics, ensemble, tree, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, classification_report, cohen_kappa_score, f1_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Credenciales
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
#TEST
drosophilaTest = gc.open('Drosophila_test').sheet1
rows_test = drosophilaTest.get_all_values()
df_test = pd.DataFrame(rows_test)
df_test.columns = df_test.iloc[0]
df_test = df_test.iloc[1:]

#TRAIN
drosophilaTrain = gc.open('Drosophila_train').sheet1
rows_train = drosophilaTrain.get_all_values()
df_train = pd.DataFrame(rows_train)
df_train.columns = df_train.iloc[0]
df_train = df_train.iloc[1:]

In [ ]:
#DATOS
Y_train = df_train["target"]
X_train = df_train.loc[:, 'S1':'S663']

df_train.columns

Index(['ID', 'target', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8',
       ...
       'S654', 'S655', 'S656', 'S657', 'S658', 'S659', 'S660', 'S661', 'S662',
       'S663'],
      dtype='object', name=0, length=665)

In [ ]:
#Procesamiento de datos
df = df_train.loc[:, 'target':'S663']
df_X = df_train.loc[:, 'S1':'S663']

one_hot_encoded_data = pd.get_dummies(df, columns = df_X.columns)

In [ ]:
#Limpiamos las filas adicionales
#Se va a usar cinco veces
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]
one_hot_encoded_data = one_hot_encoded_data.iloc[:-1, :]

In [ ]:
#Pasamos los nombres por etiquetas numericas
le = preprocessing.LabelEncoder()
le.fit(one_hot_encoded_data.target)

list(le.classes_)
one_hot_encoded_data['target'] = le.transform(one_hot_encoded_data.target)


le.inverse_transform(one_hot_encoded_data['target'])
target = one_hot_encoded_data['target']
one_hot_encoded_data_x = one_hot_encoded_data.loc[:, 'S1_A':'S663_T']
x_train, x_test, y_train, y_test = model_selection.train_test_split(one_hot_encoded_data_x, target, test_size = 0.3, random_state =0)

#Tamaño de los archivos
print("Tamaño del archivo de test:", x_test.shape)
print("Tamaño del archivo para train:", x_train.shape)

Tamaño del archivo de test: (150, 3315)
Tamaño del archivo para train: (349, 3315)


In [ ]:
#ENTRENAMIENTO USANDO RANDOM FOREST

clasificador = ensemble.RandomForestClassifier()
modelo = clasificador.fit(x_train,y_train)
print("Sociedad arácnida presenta una identificación de especies a partir de secuencias de ADN usando el algoritmo de machine learning Random Forest:",modelo)


print("\n\nValores de x y su puntuación \n", x_test)
modelo.score(x_test,y_test)

Sociedad arácnida presenta una identificación de especies a partir de secuencias de ADN usando el algoritmo de machine learning Random Forest: RandomForestClassifier()


Valores de x y su puntuación 
      S1_A  S1_C  S1_G  S1_N  S1_T  S2_A  S2_C  S2_G  S2_N  S2_T  ...  S662_A  \
91      0     0     0     1     0     0     0     0     1     0  ...       0   
255     1     0     0     0     0     0     0     0     0     1  ...       0   
284     0     0     0     1     0     0     0     0     1     0  ...       0   
445     0     0     0     1     0     0     0     0     1     0  ...       0   
475     0     0     0     1     0     0     0     0     1     0  ...       0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...     ...   
5       1     0     0     0     0     0     0     0     0     1  ...       0   
319     0     0     0     1     0     0     0     0     1     0  ...       0   
331     0     0     0     1     0     0     0     0     1     0  ...       0   

0.9733333333333334

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(one_hot_encoded_data_x, target, test_size=0.3, random_state=43)
y_pred = modelo.predict(x_test)
report = metrics.classification_report(y_test,y_pred)
print(classification_report(y_test,y_pred))

print("Precisión del modelo: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("Kappa score: {:.2f}%".format(cohen_kappa_score(y_test,y_pred)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00        10
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00        12
          11       1.00      1.00      1.00        10
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         5
          14       1.00      1.00      1.00        16
          15       0.93      1.00      0.96        25
          16       1.00      1.00      1.00         6
          17       1.00      0.95      0.98        43
          18       1.00    

In [ ]:
le.inverse_transform(y_pred)

array(['Drosophila_arizonae', 'Drosophila_mojavensis',
       'Drosophila_simulans', 'Drosophila_melanogaster',
       'Drosophila_innubila', 'Drosophila_recens', 'Drosophila_recens',
       'Drosophila_recens', 'Drosophila_pachea', 'Drosophila_innubila',
       'Drosophila_subquinaria', 'Drosophila_subquinaria',
       'Drosophila_nigrospiracula', 'Drosophila_montana',
       'Drosophila_innubila', 'Drosophila_subquinaria',
       'Drosophila_recens', 'Drosophila_subquinaria',
       'Drosophila_mojavensis', 'Drosophila_subquinaria',
       'Drosophila_montana', 'Drosophila_innubila',
       'Drosophila_subquinaria', 'Drosophila_subquinaria',
       'Drosophila_virilis', 'Drosophila_recens',
       'Drosophila_nigrospiracula', 'Drosophila_arizonae',
       'Drosophila_montana', 'Drosophila_recens', 'Drosophila_recens',
       'Drosophila_mojavensis', 'Drosophila_innubila',
       'Drosophila_subquinaria', 'Drosophila_mettleri',
       'Drosophila_pachea', 'Drosophila_simulans',
      

In [ ]:
le.inverse_transform(y_test)

array(['Drosophila_arizonae', 'Drosophila_mojavensis',
       'Drosophila_simulans', 'Drosophila_melanogaster',
       'Drosophila_innubila', 'Drosophila_recens', 'Drosophila_recens',
       'Drosophila_recens', 'Drosophila_pachea', 'Drosophila_innubila',
       'Drosophila_subquinaria', 'Drosophila_subquinaria',
       'Drosophila_nigrospiracula', 'Drosophila_montana',
       'Drosophila_innubila', 'Drosophila_subquinaria',
       'Drosophila_recens', 'Drosophila_subquinaria',
       'Drosophila_mojavensis', 'Drosophila_subquinaria',
       'Drosophila_montana', 'Drosophila_innubila',
       'Drosophila_subquinaria', 'Drosophila_subquinaria',
       'Drosophila_virilis', 'Drosophila_recens',
       'Drosophila_nigrospiracula', 'Drosophila_arizonae',
       'Drosophila_montana', 'Drosophila_recens', 'Drosophila_recens',
       'Drosophila_mojavensis', 'Drosophila_innubila',
       'Drosophila_subquinaria', 'Drosophila_mettleri',
       'Drosophila_pachea', 'Drosophila_simulans',
      

In [ ]:
pred_df = pd.DataFrame()
pred_df['Esp'] = le.inverse_transform(y_pred)
pred_df['Obs'] = le.inverse_transform(y_test)
print(pred_df)

                           Esp                        Obs
0          Drosophila_arizonae        Drosophila_arizonae
1        Drosophila_mojavensis      Drosophila_mojavensis
2          Drosophila_simulans        Drosophila_simulans
3      Drosophila_melanogaster    Drosophila_melanogaster
4          Drosophila_innubila        Drosophila_innubila
..                         ...                        ...
145          Drosophila_recens          Drosophila_recens
146  Drosophila_nigrospiracula  Drosophila_nigrospiracula
147         Drosophila_falleni         Drosophila_falleni
148     Drosophila_subquinaria     Drosophila_subquinaria
149     Drosophila_subquinaria     Drosophila_subquinaria

[150 rows x 2 columns]


In [ ]:
#One-hot para las pruebas
df_X_test = df_test.loc[:, 'S1':'S663']
one_hot_encoded_test = pd.get_dummies(df_X_test, columns = df_X_test.columns)

one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]
one_hot_encoded_test = one_hot_encoded_test.iloc[:-1, :]

one_hot_encoded_test = one_hot_encoded_test[x_train.columns]

pred = modelo.predict(one_hot_encoded_test)

In [ ]:
le.inverse_transform(pred)

array(['Drosophila_mojavensis', 'Drosophila_angor', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_barutani', 'Drosophila_falleni', 'Drosophila_falleni',
       'Drosophila_falleni', 'Drosophila_innubila', 'Drosophila_innubila',
       'Drosophila_innubila', 'Drosophila_innubila',
       'Drosophila_melanogaster', 'Drosophila_melanogaster',
       'Drosophila_mettleri', 'Drosophila_mettleri',
       'Drosophila_mettleri', 'Drosophila_mettleri',
       'Drosophila_mettleri', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_montana', 'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_montana', 'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_nigr

In [ ]:
#RESULTADO
df_test = df_test.iloc[:-1, :]
df_test = df_test.iloc[:-1, :]
df_test = df_test.iloc[:-1, :]
df_test = df_test.iloc[:-1, :]
df_test = df_test.iloc[:-1, :]

df_test['Organismo'] = le.inverse_transform(pred)
df_test['Etiquetas'] = pred
print(df_test[['ID','Organismo','Etiquetas']])

0                    ID               Organismo  Etiquetas
1    DQ471543_110189412   Drosophila_mojavensis         10
2    DQ471554_110189434        Drosophila_angor          0
3     DQ383671_87475141     Drosophila_arizonae          1
4     DQ383677_87475153     Drosophila_arizonae          1
5     DQ383678_87475155     Drosophila_arizonae          1
..                  ...                     ...        ...
112  DQ851680_114187161  Drosophila_subquinaria         17
113  DQ851686_114187173  Drosophila_subquinaria         17
114  DQ851689_114187179  Drosophila_subquinaria         17
115   DQ426800_90018979      Drosophila_virilis         18
116   DQ426803_90018985      Drosophila_virilis         18

[116 rows x 3 columns]


In [ ]:
#EXPORTACIÓN DE LOS DATOS A GOOGLE SHEETS
titulo = 'Identificación de especies a partir de secuencias de ADN mediante Machine Learning'
gc.create(titulo)
sheet = gc.open(titulo).sheet1
set_with_dataframe(sheet, df_test[['ID','Organismo','Etiquetas']])